In [1]:
import utils; reload(utils)
from utils import plots

Using cuDNN version 5103 on context None
Preallocating 10867/11439 Mb (0.950000) on cuda
Mapped name None to device cuda: Tesla K40c (0000:81:00.0)
Using Theano backend.


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [3]:
path = "data/fisheries/sample/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

# batch_size=100
batch_size=4

In [4]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [5]:
val_batches = get_batches(path+'valid', shuffle = False, batch_size=batch_size)
train_batches = get_batches(path+'train', shuffle = False, batch_size=batch_size)

Found 213 images belonging to 8 classes.
Found 307 images belonging to 8 classes.


In [6]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [7]:
train_data = get_data(path+'train')
val_data = get_data(path+'valid')

Found 307 images belonging to 8 classes.
Found 213 images belonging to 8 classes.


In [8]:
save_array(model_path+'train_data.bc',train_data)
save_array(model_path+'val_data.bc',val_data)

In [9]:
train_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'val_data.bc')

In [10]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [11]:
train_classes = train_batches.classes
train_labels = onehot(train_classes)
val_classes = val_batches.classes
val_labels = onehot(val_classes)

In [12]:
train_features = model.predict(train_data,batch_size=batch_size)
val_features = model.predict(train_data,batch_size=batch_size)

In [13]:
train_features.shape

(307, 1000)

In [14]:
save_array(model_path+'train_lastlayer_features.bc',train_features)
save_array(model_path+'val_lastlayer_features.bc',val_features)

In [15]:
train_features = load_array(model_path+'train_lastlayer_features.bc')
val_features = load_array(model_path+'val_lastlayer_features.bc')

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [17]:
sgd = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)

In [18]:
myModel = Sequential()

In [19]:
train_data.shape

(307, 3, 224, 224)

In [20]:
myModel.add(BatchNormalization(axis=1,input_shape=(3,224,224)))
myModel.add(Convolution2D(3,3,32,activation='relu'))
myModel.add(BatchNormalization(axis=1))
myModel.add(MaxPooling2D(pool_size=(2,2)))
myModel.add(Flatten())
myModel.add(Dense(8,activation='softmax', W_regularizer=l2(0.01)))

In [21]:
myModel.compile(sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
myModel.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, 
                      validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
307/307 [==============================] - 12s - loss: 1.7846 - acc: 0.5765 - val_loss: 6.6033 - val_acc: 0.1831
Epoch 2/2
307/307 [==============================] - 11s - loss: 1.2089 - acc: 0.6743 - val_loss: 3.7177 - val_acc: 0.2113


Major Key: SGD did way better than Adam on this

In [23]:
myModel.optimizer.lr=1e-3

In [24]:
myModel.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2, 
                      validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
307/307 [==============================] - 11s - loss: 1.0734 - acc: 0.7199 - val_loss: 1.9440 - val_acc: 0.4883
Epoch 2/2
307/307 [==============================] - 11s - loss: 0.7735 - acc: 0.8274 - val_loss: 1.7796 - val_acc: 0.4883


Increased lr by a factor of 10--went even better! 60.56% on validation but 97% on training... underfitting???

In [25]:
from keras.optimizers import Nadam

In [26]:
myModel.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 3, 222, 193)   867         batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 3, 222, 193)   12          convolution2d_14[0][0]           
____________________________________________________________________________________________________
maxpooling2d_6 (MaxPooling2D)    (None, 3, 111, 96)    0           batchnormalization_2[0][0]       
___________________________________________________________________________________________

In [27]:
myModel.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=2,
                     validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/2
307/307 [==============================] - 11s - loss: 0.6615 - acc: 0.8697 - val_loss: 1.8520 - val_acc: 0.5634
Epoch 2/2
307/307 [==============================] - 11s - loss: 0.5587 - acc: 0.9055 - val_loss: 1.7588 - val_acc: 0.5775


Let's try some data augmentation! I'll also now try 10 epochs and see if things improve.

In [28]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 307 images belonging to 8 classes.


In [29]:
myModel = Sequential([
    BatchNormalization(axis=1,input_shape=(3,224,224)),
    Convolution2D(3,3,32,activation='relu'),
    BatchNormalization(axis=1),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(8,activation='softmax', W_regularizer=l2(0.01))
])



newModel = Sequential([
    BatchNormalization(axis=1,input_shape=(3,224,224)),
    Convolution2D(3,3,32,activation='relu'),
    BatchNormalization(axis=1),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(8,activation='softmax', W_regularizer=l2(0.01))
])

In [37]:
def nadam1(batches):
    model = Sequential([
        BatchNormalization(axis=1,input_shape=(3,224,224)),
        Convolution2D(3,3,32,activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(8,activation='softmax', W_regularizer=l2(0.01))
    ])

    model.compile(Nadam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [38]:
# print("Nadam:")
# myModel.compile(Nadam, loss='categorical_crossentropy',metrics=['accuracy'])
# myModel.fit_generator(batches, batches.nb_sample, nb_epoch=10, 
#                     validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

print("Nadam w slower rate, 5 epochs")
model = nadam1(batches)

Nadam w slower rate, 5 epochs
Epoch 1/10
307/307 [==============================] - 12s - loss: 2.5352 - acc: 0.3811 - val_loss: 2.7860 - val_acc: 0.4648
Epoch 2/10
307/307 [==============================] - 11s - loss: 1.9833 - acc: 0.4919 - val_loss: 2.4024 - val_acc: 0.4695
Epoch 3/10
307/307 [==============================] - 11s - loss: 1.6503 - acc: 0.5700 - val_loss: 2.1790 - val_acc: 0.4225
Epoch 4/10
307/307 [==============================] - 11s - loss: 1.4354 - acc: 0.6384 - val_loss: 2.1610 - val_acc: 0.4930
Epoch 5/10
307/307 [==============================] - 11s - loss: 1.3813 - acc: 0.6319 - val_loss: 2.0069 - val_acc: 0.5117
Epoch 6/10
307/307 [==============================] - 11s - loss: 1.3305 - acc: 0.6645 - val_loss: 2.3662 - val_acc: 0.4742
Epoch 7/10
307/307 [==============================] - 11s - loss: 1.1625 - acc: 0.7068 - val_loss: 1.8504 - val_acc: 0.5822
Epoch 8/10
307/307 [==============================] - 11s - loss: 0.9239 - acc: 0.7590 - val_loss: 1.9

Data augmentation with everything!

In [39]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.05, 
                                shear_range=0.1, rotation_range=15, channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 307 images belonging to 8 classes.


In [40]:
model = nadam1(batches)

Epoch 1/10
307/307 [==============================] - 12s - loss: 2.4586 - acc: 0.3876 - val_loss: 3.0396 - val_acc: 0.3427
Epoch 2/10
307/307 [==============================] - 11s - loss: 2.1106 - acc: 0.4658 - val_loss: 2.1184 - val_acc: 0.4789
Epoch 3/10
307/307 [==============================] - 11s - loss: 1.9526 - acc: 0.5081 - val_loss: 2.2154 - val_acc: 0.4695
Epoch 4/10
307/307 [==============================] - 11s - loss: 1.9679 - acc: 0.5277 - val_loss: 2.2608 - val_acc: 0.4554
Epoch 5/10
307/307 [==============================] - 11s - loss: 1.9388 - acc: 0.4853 - val_loss: 2.0875 - val_acc: 0.5023
Epoch 6/10
307/307 [==============================] - 11s - loss: 1.9619 - acc: 0.5277 - val_loss: 2.2217 - val_acc: 0.5070
Epoch 7/10
307/307 [==============================] - 11s - loss: 1.8347 - acc: 0.5537 - val_loss: 2.3904 - val_acc: 0.4930
Epoch 8/10
307/307 [==============================] - 11s - loss: 1.8023 - acc: 0.5635 - val_loss: 2.2147 - val_acc: 0.5258
Epoch 9/

It's getting better--let's try turning down lr and running more epochs!

Findings:
We keep getting better! Epoch 5 of Nadam w a slower rate seems to be best, but it looks like we're overfitting on the training data a little bit... Let's come back and see what we can do to reduce this overfitting. 

Also, for some reason when I take things out of functions it starts giving me errors. I can't use model.optimizer.lr when I have model as a function but when I take it out I get a host of other errors.

I'm also getting some cases where as the epoch continues the accuracy gets worse... Also in this specific case my val_loss was the highest on the final epoch.

In [41]:
model.optimizer.lr = 0.0001

In [42]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/10
307/307 [==============================] - 12s - loss: 1.4011 - acc: 0.6678 - val_loss: 2.4876 - val_acc: 0.5540
Epoch 2/10
307/307 [==============================] - 11s - loss: 1.4969 - acc: 0.6482 - val_loss: 2.3973 - val_acc: 0.5634
Epoch 3/10
307/307 [==============================] - 11s - loss: 1.5251 - acc: 0.6254 - val_loss: 2.6143 - val_acc: 0.4977
Epoch 4/10
307/307 [==============================] - 11s - loss: 1.4222 - acc: 0.6124 - val_loss: 2.5558 - val_acc: 0.5587
Epoch 5/10
307/307 [==============================] - 11s - loss: 1.6291 - acc: 0.6059 - val_loss: 2.6840 - val_acc: 0.5446
Epoch 6/10
307/307 [==============================] - 11s - loss: 1.5204 - acc: 0.6319 - val_loss: 2.4550 - val_acc: 0.5352
Epoch 7/10
307/307 [==============================] - 11s - loss: 1.3750 - acc: 0.6384 - val_loss: 2.2908 - val_acc: 0.5681
Epoch 8/10
307/307 [==============================] - 11s - loss: 1.3818 - acc: 0.6710 - val_loss: 2.6775 - val_acc: 0.5915
Epoch 9/

What I know:
We're most likely underfitting--val accuracy/loss gets worse as time goes on sometimes, it doesn't seem like there's a consistent improvement or decline though. I've added both data aug and L2 regularization, so I'm not sure what other ways might help with what I'm trying to do. Maybe remove L2 since it technically prevents ~overfitting?

Let's also try less data aug.

In [43]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.05, 
                                shear_range=0.1, channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 307 images belonging to 8 classes.


In [44]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/10
307/307 [==============================] - 12s - loss: 1.1075 - acc: 0.6906 - val_loss: 2.3134 - val_acc: 0.5822
Epoch 2/10
307/307 [==============================] - 11s - loss: 1.3541 - acc: 0.6710 - val_loss: 2.5433 - val_acc: 0.5164
Epoch 3/10
307/307 [==============================] - 11s - loss: 1.3007 - acc: 0.6580 - val_loss: 2.4596 - val_acc: 0.5775
Epoch 4/10
307/307 [==============================] - 11s - loss: 1.3090 - acc: 0.6580 - val_loss: 2.5375 - val_acc: 0.5634
Epoch 5/10
307/307 [==============================] - 11s - loss: 1.2172 - acc: 0.6775 - val_loss: 2.4922 - val_acc: 0.5493
Epoch 6/10
307/307 [==============================] - 11s - loss: 1.1170 - acc: 0.6971 - val_loss: 2.5441 - val_acc: 0.5540
Epoch 7/10
307/307 [==============================] - 11s - loss: 1.3022 - acc: 0.6678 - val_loss: 2.6452 - val_acc: 0.5587
Epoch 8/10
307/307 [==============================] - 11s - loss: 1.1414 - acc: 0.7231 - val_loss: 2.6676 - val_acc: 0.5728
Epoch 9/

In [45]:
def nadam2(batches):
    #same as before, just without L2!
    model = Sequential([
        BatchNormalization(axis=1,input_shape=(3,224,224)),
        Convolution2D(3,3,32,activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(8,activation='softmax')
    ])

    model.compile(Nadam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [46]:
model2 = nadam2(batches)

Epoch 1/10
307/307 [==============================] - 12s - loss: 2.2928 - acc: 0.3909 - val_loss: 2.5572 - val_acc: 0.4413
Epoch 2/10
307/307 [==============================] - 11s - loss: 1.9985 - acc: 0.4756 - val_loss: 2.1686 - val_acc: 0.3709
Epoch 3/10
307/307 [==============================] - 11s - loss: 1.5476 - acc: 0.5700 - val_loss: 2.1391 - val_acc: 0.4460
Epoch 4/10
307/307 [==============================] - 11s - loss: 1.8373 - acc: 0.4886 - val_loss: 2.7411 - val_acc: 0.3709
Epoch 5/10
307/307 [==============================] - 11s - loss: 1.6762 - acc: 0.5570 - val_loss: 2.3732 - val_acc: 0.4789
Epoch 6/10
307/307 [==============================] - 11s - loss: 1.5732 - acc: 0.6091 - val_loss: 2.1126 - val_acc: 0.5728
Epoch 7/10
307/307 [==============================] - 11s - loss: 1.2451 - acc: 0.6515 - val_loss: 2.2953 - val_acc: 0.5070
Epoch 8/10
307/307 [==============================] - 11s - loss: 1.6839 - acc: 0.5961 - val_loss: 2.3208 - val_acc: 0.4836
Epoch 9/

# Baseline: Simple Linear Model

In [18]:
model = Sequential([
        BatchNormalization(axis=1,input_shape=(3,224,224)),
        Flatten(),
        Dense(8,activation='softmax')
    ])

In [20]:
model.compile('Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, train_batches.nb_sample,nb_epoch=3, validation_data = val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
307/307 [==============================] - 13s - loss: 9.2871 - acc: 0.4007 - val_loss: 8.7779 - val_acc: 0.4554
Epoch 2/3
307/307 [==============================] - 11s - loss: 8.3692 - acc: 0.4756 - val_loss: 9.6039 - val_acc: 0.3991
Epoch 3/3
307/307 [==============================] - 12s - loss: 8.0328 - acc: 0.5016 - val_loss: 9.5357 - val_acc: 0.4085


In [21]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 3, 224, 224)   12          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 150528)        0           batchnormalization_1[0][0]       
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 8)             1204232     flatten_2[0][0]                  
Total params: 1,204,244
Trainable params: 1,204,238
Non-trainable params: 6
____________________________________________________________________________________________________


In [22]:
np.round(model.predict_generator(train_batches, train_batches.N)[:10],2)

AttributeError: 'DirectoryIterator' object has no attribute 'N'

Let's try a lower learning rate!

In [23]:
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
307/307 [==============================] - 12s - loss: 8.0328 - acc: 0.5016 - val_loss: 9.5571 - val_acc: 0.4038
Epoch 2/3
307/307 [==============================] - 11s - loss: 8.0328 - acc: 0.5016 - val_loss: 9.5918 - val_acc: 0.4038
Epoch 3/3
307/307 [==============================] - 11s - loss: 8.0328 - acc: 0.5016 - val_loss: 9.5874 - val_acc: 0.4038


Huh, val_acc = 0.4038 for every one of the above--that's interesting consistency.

In [24]:
model.optimizer.lr = 0.001
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
307/307 [==============================] - 12s - loss: 8.0328 - acc: 0.5016 - val_loss: 9.5942 - val_acc: 0.3991
Epoch 2/3
307/307 [==============================] - 11s - loss: 8.0328 - acc: 0.5016 - val_loss: 9.6017 - val_acc: 0.3991
Epoch 3/3
307/307 [==============================] - 12s - loss: 8.0328 - acc: 0.5016 - val_loss: 9.6053 - val_acc: 0.3991


Again, acc=0.5016 the same as before, while val_acc is lower but only slightly so at 0.3991--but they're all the same again!! So we've hit some sort of consistency but we're still probably at either some local minimum or jumping too far to hit the global?S

In [25]:
rnd_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=True)

Found 213 images belonging to 8 classes.


In [26]:
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.nb_sample) for i in range(10)]
np.round(val_res,2)

array([[ 9.61,  0.4 ],
       [ 9.23,  0.42],
       [ 9.45,  0.41],
       [ 9.91,  0.38],
       [ 9.91,  0.38],
       [ 9.76,  0.39],
       [ 9.53,  0.4 ],
       [ 9.45,  0.41],
       [ 9.83,  0.38],
       [ 9.53,  0.4 ]])

From StackOverflow: evaluate_generator uses both your test input and output. It first predicts output using training input and then evaluates performance by comparing it against your test output. So it gives out a measure of performance, i.e. accuracy in your case.

So what we just did above was see what the performance of our model is on 10 things so we can see if there are statstically significant differences in performance. 

Turns out it's pretty consistent:

min_acc = 0.38
max_acc = 0.42

# Linear w/ L2 Regularization

In [27]:
model = Sequential([
        BatchNormalization(axis=1,input_shape=(3,224,224)),
        Flatten(),
        Dense(8,activation='softmax', W_regularizer=l2(0.01))
    ])

model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
307/307 [==============================] - 13s - loss: 11.0260 - acc: 0.1889 - val_loss: 12.8843 - val_acc: 0.1784
Epoch 2/3
307/307 [==============================] - 11s - loss: 9.8730 - acc: 0.3453 - val_loss: 12.3482 - val_acc: 0.1737
Epoch 3/3
307/307 [==============================] - 11s - loss: 7.8282 - acc: 0.4788 - val_loss: 9.7379 - val_acc: 0.3709


So that got... worse? Except for the last val_acc=.3709 which seems close to what we had before but now val_acc is around .17

In [28]:
model.optimizer.lr=0.001

In [30]:
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
307/307 [==============================] - 12s - loss: 6.9488 - acc: 0.5537 - val_loss: 8.8606 - val_acc: 0.4085
Epoch 2/3
307/307 [==============================] - 11s - loss: 6.4381 - acc: 0.5798 - val_loss: 7.6627 - val_acc: 0.5023
Epoch 3/3
307/307 [==============================] - 13s - loss: 5.8013 - acc: 0.6287 - val_loss: 7.6443 - val_acc: 0.5258


woahhhh we jumped like 10 whole percents

# Single layer!

In [31]:
model = Sequential([
        BatchNormalization(axis=1,input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(8,activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
307/307 [==============================] - 12s - loss: 2.4102 - acc: 0.1107 - val_loss: 5.9550 - val_acc: 0.0516
Epoch 2/3
307/307 [==============================] - 11s - loss: 2.2192 - acc: 0.1433 - val_loss: 3.3458 - val_acc: 0.0704
Epoch 3/3
307/307 [==============================] - 11s - loss: 2.1259 - acc: 0.1759 - val_loss: 2.7295 - val_acc: 0.0939


# Full Dataset?

Let's try our models on the full dataset to see wtf happens

In [11]:
#redefine location:
path = "data/fisheries/"
batch_size=64

In [12]:
val_batches = get_batches(path+'valid', shuffle = False, batch_size=batch_size)
train_batches = get_batches(path+'train', shuffle = False, batch_size=batch_size)

Found 756 images belonging to 8 classes.
Found 3021 images belonging to 8 classes.


In [13]:
train_data = get_data(path+'train')
val_data = get_data(path+'valid')

Found 3021 images belonging to 8 classes.
Found 756 images belonging to 8 classes.


In [14]:
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [15]:
save_array(model_path+'train_data.bc',train_data)
save_array(model_path+'val_data.bc',val_data)

In [16]:
train_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'val_data.bc')

In [17]:
train_classes = train_batches.classes
train_labels = onehot(train_classes)
val_classes = val_batches.classes
val_labels = onehot(val_classes)

In [18]:
model = Sequential([
        BatchNormalization(axis=1,input_shape=(3,224,224)),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(8,activation='softmax')
    ])
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=5, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/5
3021/3021 [==============================] - 113s - loss: 2.3006 - acc: 0.1026 - val_loss: 6.8776 - val_acc: 0.1085
Epoch 2/5
3021/3021 [==============================] - 90s - loss: 2.1338 - acc: 0.1387 - val_loss: 4.0829 - val_acc: 0.0979
Epoch 3/5
3021/3021 [==============================] - 80s - loss: 2.0865 - acc: 0.1625 - val_loss: 2.4723 - val_acc: 0.1362
Epoch 4/5
3021/3021 [==============================] - 76s - loss: 2.0624 - acc: 0.1801 - val_loss: 2.1891 - val_acc: 0.1772
Epoch 5/5
3021/3021 [==============================] - 77s - loss: 2.0405 - acc: 0.1993 - val_loss: 1.9989 - val_acc: 0.2235


In [19]:
model.optimizer.lr=0.001
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=5, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/5
3021/3021 [==============================] - 88s - loss: 2.0324 - acc: 0.2171 - val_loss: 1.9101 - val_acc: 0.2540
Epoch 2/5
3021/3021 [==============================] - 74s - loss: 2.0216 - acc: 0.2122 - val_loss: 1.8664 - val_acc: 0.2844
Epoch 3/5
3021/3021 [==============================] - 73s - loss: 2.0090 - acc: 0.2105 - val_loss: 1.8136 - val_acc: 0.3280
Epoch 4/5
3021/3021 [==============================] - 74s - loss: 1.9979 - acc: 0.2387 - val_loss: 1.7976 - val_acc: 0.3836
Epoch 5/5
3021/3021 [==============================] - 74s - loss: 1.9893 - acc: 0.2483 - val_loss: 1.7904 - val_acc: 0.3730


Let's try a different batch size.

In [20]:
batch_size=32

In [21]:
val_batches = get_batches(path+'valid', shuffle = False, batch_size=batch_size)
train_batches = get_batches(path+'train', shuffle = False, batch_size=batch_size)

Found 756 images belonging to 8 classes.
Found 3021 images belonging to 8 classes.


ok we need to actually do smth w batch size

In [22]:
val_batches = get_batches(path+'valid', shuffle = False, batch_size=batch_size)
train_batches = get_batches(path+'train', shuffle = False, batch_size=batch_size)

Found 756 images belonging to 8 classes.
Found 3021 images belonging to 8 classes.


In [23]:
model.optimizer.lr=0.001
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3021/3021 [==============================] - 86s - loss: 2.0613 - acc: 0.1837 - val_loss: 1.7948 - val_acc: 0.3611
Epoch 2/3
3021/3021 [==============================] - 79s - loss: 2.0496 - acc: 0.1870 - val_loss: 1.7940 - val_acc: 0.3505
Epoch 3/3
3021/3021 [==============================] - 78s - loss: 2.0377 - acc: 0.1903 - val_loss: 1.8203 - val_acc: 0.3148


# Data Augmentation

In [24]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.05, 
                                shear_range=0.1, rotation_range=15, channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 3021 images belonging to 8 classes.


In [25]:
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3021/3021 [==============================] - 86s - loss: 1.7179 - acc: 0.4406 - val_loss: 1.2769 - val_acc: 0.6389
Epoch 2/3
3021/3021 [==============================] - 81s - loss: 1.4685 - acc: 0.5425 - val_loss: 1.1220 - val_acc: 0.6786
Epoch 3/3
3021/3021 [==============================] - 83s - loss: 1.3954 - acc: 0.5746 - val_loss: 1.0174 - val_acc: 0.7196


In [26]:
model.optimizer.lr = 0.001
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3021/3021 [==============================] - 86s - loss: 1.3066 - acc: 0.6001 - val_loss: 0.9981 - val_acc: 0.7050
Epoch 2/3
3021/3021 [==============================] - 79s - loss: 1.3039 - acc: 0.5995 - val_loss: 1.0082 - val_acc: 0.7183
Epoch 3/3
3021/3021 [==============================] - 78s - loss: 1.2326 - acc: 0.6379 - val_loss: 0.8996 - val_acc: 0.7474


Let's try yet another batch size!

In [27]:
batch_size=4
val_batches = get_batches(path+'valid', shuffle = False, batch_size=batch_size)
train_batches = get_batches(path+'train', shuffle = False, batch_size=batch_size)

Found 756 images belonging to 8 classes.
Found 3021 images belonging to 8 classes.


In [28]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.05, 
                                shear_range=0.1, rotation_range=15, channel_shift_range=20)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 3021 images belonging to 8 classes.


In [29]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3021/3021 [==============================] - 84s - loss: 1.7100 - acc: 0.4472 - val_loss: 1.2677 - val_acc: 0.6257
Epoch 2/3
3021/3021 [==============================] - 87s - loss: 1.6613 - acc: 0.4489 - val_loss: 1.1852 - val_acc: 0.6706
Epoch 3/3
3021/3021 [==============================] - 84s - loss: 1.6243 - acc: 0.4641 - val_loss: 1.2041 - val_acc: 0.6429


In [30]:
model.optimizer.lr = 0.001
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3021/3021 [==============================] - 87s - loss: 1.5894 - acc: 0.4869 - val_loss: 1.1775 - val_acc: 0.6429
Epoch 2/3
3021/3021 [==============================] - 91s - loss: 1.5771 - acc: 0.4796 - val_loss: 1.2394 - val_acc: 0.6336
Epoch 3/3
3021/3021 [==============================] - 85s - loss: 1.5593 - acc: 0.4856 - val_loss: 1.1184 - val_acc: 0.6521


# Multiple Conv Layers, No Dropout

In [31]:
model = Sequential([
    BatchNormalization(axis=1,input_shape=(3,224,224)),
    Convolution2D(32,3,3,activation='relu'),
    BatchNormalization(axis=1),
    MaxPooling2D((3,3)),
    Convolution2D(64,3,3,activation='relu'),
    BatchNormalization(axis=1),
    MaxPooling2D((3,3)),
    Flatten(),
    Dense(8,activation='softmax')
])

In [32]:
model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)
model.optimizer.lr = 0.001
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3021/3021 [==============================] - 85s - loss: 5.7406 - acc: 0.6319 - val_loss: 11.6924 - val_acc: 0.2407
Epoch 2/3
3021/3021 [==============================] - 84s - loss: 9.2482 - acc: 0.4035 - val_loss: 12.6926 - val_acc: 0.2050
Epoch 3/3
3021/3021 [==============================] - 84s - loss: 7.7163 - acc: 0.4975 - val_loss: 8.3025 - val_acc: 0.4153
Epoch 1/3
3021/3021 [==============================] - 84s - loss: 6.3223 - acc: 0.5885 - val_loss: 10.1639 - val_acc: 0.3280
Epoch 2/3
3021/3021 [==============================] - 83s - loss: 5.3827 - acc: 0.6485 - val_loss: 9.4309 - val_acc: 0.3545
Epoch 3/3
3021/3021 [==============================] - 83s - loss: 5.6896 - acc: 0.6236 - val_loss: 11.1571 - val_acc: 0.2632


Now with augmented data!

In [33]:
model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)
model.optimizer.lr = 0.001
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3021/3021 [==============================] - 87s - loss: 6.5129 - acc: 0.5081 - val_loss: 4.5715 - val_acc: 0.6548
Epoch 2/3
3021/3021 [==============================] - 86s - loss: 4.8140 - acc: 0.5770 - val_loss: 4.6160 - val_acc: 0.5979
Epoch 3/3
3021/3021 [==============================] - 84s - loss: 4.3397 - acc: 0.6071 - val_loss: 3.0918 - val_acc: 0.7063
Epoch 1/3
3021/3021 [==============================] - 85s - loss: 3.8087 - acc: 0.6200 - val_loss: 3.2387 - val_acc: 0.7011
Epoch 2/3
3021/3021 [==============================] - 85s - loss: 3.3493 - acc: 0.6352 - val_loss: 2.8213 - val_acc: 0.7143
Epoch 3/3
3021/3021 [==============================] - 84s - loss: 2.9250 - acc: 0.6614 - val_loss: 2.1025 - val_acc: 0.7646


# Stanford-Recommended Model

In [34]:
model = Sequential([
    BatchNormalization(axis=1,input_shape=(3,224,224)),
    Convolution2D(32,3,3,activation='relu'),
    BatchNormalization(axis=1),
    MaxPooling2D((3,3)),
    Convolution2D(64,3,3,activation='relu'),
    BatchNormalization(axis=1),
    MaxPooling2D((3,3)),
    Flatten(),
    Dense(100,activation='relu'),
    BatchNormalization(axis=1),
    Dense(8,activation='softmax')
])

In [35]:
model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=5, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/5
3021/3021 [==============================] - 85s - loss: 2.2835 - acc: 0.1278 - val_loss: 2.4026 - val_acc: 0.1680
Epoch 2/5
3021/3021 [==============================] - 84s - loss: 2.1379 - acc: 0.1235 - val_loss: 2.4434 - val_acc: 0.1627
Epoch 3/5
3021/3021 [==============================] - 84s - loss: 2.0888 - acc: 0.1599 - val_loss: 2.4579 - val_acc: 0.1481
Epoch 4/5
3021/3021 [==============================] - 84s - loss: 2.0530 - acc: 0.1887 - val_loss: 2.4119 - val_acc: 0.1680
Epoch 5/5
3021/3021 [==============================] - 84s - loss: 2.0260 - acc: 0.2099 - val_loss: 2.2868 - val_acc: 0.1892


In [36]:
model.optimizer.lr = 0.001
model.fit_generator(batches, batches.nb_sample, nb_epoch=3, validation_data=val_batches,
                   nb_val_samples=val_batches.nb_sample)

Epoch 1/3
3021/3021 [==============================] - 86s - loss: 1.7861 - acc: 0.3853 - val_loss: 1.7726 - val_acc: 0.4683
Epoch 2/3
3021/3021 [==============================] - 85s - loss: 1.6163 - acc: 0.4657 - val_loss: 1.4920 - val_acc: 0.5251
Epoch 3/3
3021/3021 [==============================] - 85s - loss: 1.5182 - acc: 0.5197 - val_loss: 1.7912 - val_acc: 0.5860


# Pseudo Labels

First... Let's grab ImageNet.

In [37]:
vgg = Vgg16()
model = vgg.model
#gets last convolutional layer in the model so we can grab its output shape
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [38]:
conv_model = Sequential(conv_layers)

Now we'll make a bn model with a simplified version of vgg dense layers

In [39]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128,activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128,activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(8,activation='softmax')
    ]

Now we'll create features from vgg

In [45]:
batches = get_batches(path+'train', batch_size=64, shuffle=False)
#test_batches = get_batches(path+'test_stg1', shuffle=False, batch_size=1)

Found 3021 images belonging to 8 classes.


In [46]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 3021 images belonging to 8 classes.
Found 756 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [47]:
conv_feat = conv_model.predict_generator(batches,batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches,val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches,test_batches.nb_sample)

Exception in thread Thread-64:
Traceback (most recent call last):
  File "/home/dbashir/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/dbashir/anaconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/keras/engine/training.py", line 429, in data_generator_task
    generator_output = next(self._generator)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 832, in next
    target_size=self.target_size)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 296, in load_img
    img = pil_image.open(path)
  File "/home/dbashir/anaconda2/lib/python2.7/site-packages/PIL/Image.py", line 2477, in open
    fp = builtins.open(filename, "rb")
IOError: [Errno 2] No such file or directory: 'data/fisheries/test_stg1/unknown/img_04268.jpg'



TypeError: Error when checking : data should be a Numpy array, or list/dict of Numpy arrays. Found: None...

In [ ]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [ ]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

In [ ]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [ ]:
p=0.8

In [ ]:
bn_model=Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=1e-5), loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
#data aug
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

In [ ]:
da_conv_feat = conv_model.predict_generator(da_batches,da_batches.nb_sample*5)

In [ ]:
save_array(path+'results/da_conv_feat2.dat', da_conv_feat)

In [ ]:
da_conv_feat = load_array(path+'results/da_conv_feat2.dat')

In [ ]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

In [ ]:
da_trn_labels = np.concatenate([trn_labels]*6)

Now for pseudo labeling!

In [ ]:
val_pseudo = bn_model.predict(conv_val_feat,batch_size=batch_size)

In [ ]:
comb_pseudo = np.concatenate([da_trn_labels,val_pseudo])

In [ ]:
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

To tune the model up we'll need to train the conv thing

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.01

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.0001

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

Now to load the model!

In [ ]:
bn_model.load_weights(path+'models/da_conv8_1.h5')

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size,nb_epoch=1,
            validation_data=(conv_val_feat,val_labels))

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.optimizer.lr=0.00001

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))

In [ ]:
bn_model.save_weights(path+'models/bn-ps8.h5')

# Submit

In [ ]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [ ]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval()

In [ ]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [ ]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [ ]:
subm = do_clip(preds,0.93)

In [ ]:
subm_name = path+'results/subm.gz'

In [ ]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [ ]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[4:] for a in test_filenames])
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [ ]:
FileLink(subm_name)